In [101]:
# Imports:
import matplotlib.pyplot as plt
import keras
from keras.applications import VGG16
from keras import models
from keras import layers
from keras.models import Model, Sequential
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pandas as pd
import ast
import matplotlib.image as mpimg
import cv2
from scipy.spatial.distance import cosine
from keras.models import load_model
import ast
import pickle
import json

## Features Extraction - VGG16 pre-trained

This notebook is for features extraction using the trained VGG16 model I saved in the previous notebook. I saved all the features vector for each image in a dataframe that will be used for the recommendation system.

In [9]:
model = load_model('final_model_classifier.h5') # reference the previous notebook for details

/Users/liviasalgueiro/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
features_layer (Flatten)     (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 1799      
Total params: 16,813,895
Trainable params: 16,813,895
Non-trainable params: 0
_________________________________________________________________


In [87]:
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

In [97]:
categories = ["dress", "coat", "top", "pants", "shorts", "skirt", "jumpsuit"]
train_files = []
test_files = []
val_files = []

for cat in categories:
    
    train_path = '/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/' + cat
    all_train_files = os.listdir(train_path)
    for file in all_train_files:
        if not file.startswith("."):
            train_files.append(os.path.join(train_path, file))
            
    test_path = '/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/test/' + cat
    all_test_files = os.listdir(test_path)
    for file in all_test_files:
        if not file.startswith("."):
            test_files.append(os.path.join(test_path, file))
            
    val_path = '/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/' + cat
    all_val_files = os.listdir(val_path)
    for file in all_val_files:
        if not file.startswith("."):
            val_files.append(os.path.join(val_path, file))

In [98]:
total_files = train_files + test_files + val_files

In [99]:
print(len(total_files), len(train_files), len(test_files), len(val_files))

35211 21129 7041 7041


In [100]:
# generating the features vector for EACH image in my dataset (train, val and test)

features_vectors = {}

for outfit_path in total_files:
    print(outfit_path)
    img = cv2.imread(outfit_path)
    img = cv2.resize(img,(150,150))
    img = np.reshape(img,[1,150,150,3])
    img_vector = feature_extractor.predict(img)
    features_vectors[outfit_path] = [img_vector]

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/6385895.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3273077.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3267002.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3422705.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224791087.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8336659.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8456484.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3259846.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8330228.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8986774.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dres

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3493643.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9490311.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/7454199.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9509054.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3408463.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9358040.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3266693.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9469978.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8246631.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224130189.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dres

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3354219.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9185879.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3388372.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3305056.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3293020.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9615943.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224580091.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/5730194.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9590073.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3252654.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dres

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3384987.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8923054.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3306748.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/221770075.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3352319.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3335382.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224050052.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3414665.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3404476.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/8351589.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dr

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9469986.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3364708.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/7619961.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/223320409.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/222790572.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3334659.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3377774.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/10133851.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/2880175.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/9710557.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/d

...

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/220660352.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/186520287.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/2879848.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/3175714.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/223811811.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/3351632.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/9676021.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/8907659.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/9575553.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/skirt/221990343.jpg
/Users/liviasalgueiro/

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/9758467.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/224160634.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3472696.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3442033.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/1331372.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3375413.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/9280616.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8808827.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/221960084.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3374294.

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/224160509.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/204890179.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8987434.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/2865359.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8463649.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/225210180.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/9920761.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3428937.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/220220070.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/2194

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/6206117.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3340617.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/6491187.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3243444.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3293301.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/219110217.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3365656.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3420621.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8775298.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8999173.jp

/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3388384.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/224130221.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3392445.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3279397.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/3198112.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8477522.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/223990288.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8784957.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/8490024.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/validation/jumpsuit/21623094

In [102]:
df = pd.DataFrame.from_dict(features_vectors, orient='index')
df.columns = ["vector"]

In [103]:
df.head()

,vector
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/6385895.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.006607, 0.0, 0.0, 0.0, 0.0, 0.0,..."
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3273077.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3267002.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3422705.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224791087.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [104]:
df["category"] = df.index

In [105]:
def get_category(row):
    category = row["category"].split("/")
    category = category[-2]
    return category

df["category"] = df.apply(get_category, axis=1)

In [106]:
df.category.value_counts()

top         10810
dress       10368
pants        4900
coat         3465
skirt        2376
shorts       1647
jumpsuit     1645
Name: category, dtype: int64

In [107]:
df.to_pickle("clothes_vectors.pkl")

### Implementing the dataframe with additional information about the images:

In [234]:
stores = ["neiman", "macys", "bloomingdales"]

In [235]:
original_path = '/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/Project5_Final/AWS_Files'
frames = []
for store in stores:
    store_path = os.path.join(original_path, store)
    all_files = os.listdir(store_path)
    for file in all_files:
        if "swimwear"not in file.lower():
            file_path = os.path.join(store_path, file)
            store_df = pd.read_csv(file_path)
            store_df["original_name"] = file
            frames.append(store_df)

In [236]:
clothes_df = pd.concat(frames)

In [237]:
clothes_df.drop(columns=["Unnamed: 0"], inplace=True)

In [238]:
clothes_df.head()

,product_id,product_url,store,type,name,price,description,color,brand,original_name
0,224280389,https://www.neimanmarcus.com/p/lafayette-148-new-york-manhattan-techno-stretch-twill-slim-bermud...,Neiman Marcus,Shorts,Manhattan Techno-Stretch Twill Slim Bermuda Shorts,$67.00,"Lafayette 148 New York ""Manhattan"" shorts in techno-stretch twill",BLACKSEA SALT,Lafayette 148 New York,Shorts_pages1-10.csv
1,224420188,https://www.neimanmarcus.com/p/the-upside-leo-violet-shorts-prod224420188?childItemId=NMTZHYB_&n...,Neiman Marcus,Shorts,Leo Violet Shorts,NaN,"The Upside ""Leo Violet"" shorts in leopard-print with ladder stitching detail",LEOPARD,The Upside,Shorts_pages1-10.csv
2,216940027,https://www.neimanmarcus.com/p/agolde-parker-vintage-denim-cutoff-shorts-prod216940027?childItem...,Neiman Marcus,Shorts,Parker Vintage Denim Cutoff Shorts,NaN,"AGOLDE ""Parker"" vintage-inspired denim shorts",ROCKSTEADY,AGOLDE,Shorts_pages1-10.csv
3,197672738,https://www.neimanmarcus.com/p/alice-olivia-classic-cady-shorts-white-prod197672738?childItemId=...,Neiman Marcus,Shorts,"Classic Cady Shorts, White",NaN,Alice + Olivia shorts in cady,BLACKWHITE,Alice + Olivia,Shorts_pages1-10.csv
4,218600067,https://www.neimanmarcus.com/p/rag-bone-maya-high-rise-denim-shorts-prod218600067?childItemId=NM...,Neiman Marcus,Shorts,Maya High-Rise Denim Shorts,$136.00,"Rag & Bone ""Maya"" denim shorts with heavy distressing",TABBY WITH HOLES,rag & bone,Shorts_pages1-10.csv


In [243]:
def get_category_clothes(row):
    list_of_categories = ["dress", "coat", "top", "pants", "shorts", "skirt", "jumpsuit", "jackets", "jeans"]
    cleaned_cat = None
    complete_name = row["original_name"].lower()
    for cat in list_of_categories:
        if cat in complete_name:
            cleaned_cat = cat
    
    if cleaned_cat == "jackets":
        cleaned_cat = "coat"
    if cleaned_cat == "jeans":
        cleaned_cat = "pants"
        
    return cleaned_cat

clothes_df["category"] = clothes_df.apply(get_category_clothes, axis=1)

In [252]:
df["join_name"] = df.index
df.head()

,vector,category,recommendation,join_name
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/6385895.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.006607, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.756866,/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/6385895.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3273077.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.703605,/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3273077.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3267002.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.789901,/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3267002.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3422705.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.717229,/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3422705.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224791087.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.882713,/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224791087.jpg


In [253]:
def final_name_to_join(row):
    file_name = row['join_name'].split("/")[-1]
    final_name_to_join = row["category"] + "/" + file_name
    return final_name_to_join

df["join_name"] = df.apply(final_name_to_join, axis=1)
df.head()

,vector,category,recommendation,join_name
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/6385895.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.006607, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.756866,dress/6385895.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3273077.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.703605,dress/3273077.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3267002.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.789901,dress/3267002.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/3422705.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.717229,dress/3422705.jpg
/Users/liviasalgueiro/Documents/Data Science/Metis Bootcamp/clothes/train/dress/224791087.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",dress,0.882713,dress/224791087.jpg


In [261]:
clothes_df["product_id"] = clothes_df["product_id"].astype(str)

In [263]:
clothes_df["join_name"] = clothes_df["category"] + "/" + clothes_df["product_id"] + ".jpg"
clothes_df.head()

,product_id,product_url,store,type,name,price,description,color,brand,original_name,category,join_name
0,224280389,https://www.neimanmarcus.com/p/lafayette-148-new-york-manhattan-techno-stretch-twill-slim-bermud...,Neiman Marcus,Shorts,Manhattan Techno-Stretch Twill Slim Bermuda Shorts,$67.00,"Lafayette 148 New York ""Manhattan"" shorts in techno-stretch twill",BLACKSEA SALT,Lafayette 148 New York,Shorts_pages1-10.csv,shorts,shorts/224280389.jpg
1,224420188,https://www.neimanmarcus.com/p/the-upside-leo-violet-shorts-prod224420188?childItemId=NMTZHYB_&n...,Neiman Marcus,Shorts,Leo Violet Shorts,NaN,"The Upside ""Leo Violet"" shorts in leopard-print with ladder stitching detail",LEOPARD,The Upside,Shorts_pages1-10.csv,shorts,shorts/224420188.jpg
2,216940027,https://www.neimanmarcus.com/p/agolde-parker-vintage-denim-cutoff-shorts-prod216940027?childItem...,Neiman Marcus,Shorts,Parker Vintage Denim Cutoff Shorts,NaN,"AGOLDE ""Parker"" vintage-inspired denim shorts",ROCKSTEADY,AGOLDE,Shorts_pages1-10.csv,shorts,shorts/216940027.jpg
3,197672738,https://www.neimanmarcus.com/p/alice-olivia-classic-cady-shorts-white-prod197672738?childItemId=...,Neiman Marcus,Shorts,"Classic Cady Shorts, White",NaN,Alice + Olivia shorts in cady,BLACKWHITE,Alice + Olivia,Shorts_pages1-10.csv,shorts,shorts/197672738.jpg
4,218600067,https://www.neimanmarcus.com/p/rag-bone-maya-high-rise-denim-shorts-prod218600067?childItemId=NM...,Neiman Marcus,Shorts,Maya High-Rise Denim Shorts,$136.00,"Rag & Bone ""Maya"" denim shorts with heavy distressing",TABBY WITH HOLES,rag & bone,Shorts_pages1-10.csv,shorts,shorts/218600067.jpg


In [282]:
for file in df["join_name"]:
    if file not in list(clothes_df["join_name"]):
        print(file)

In [301]:
clothes_df.drop_duplicates(subset = ["product_id", "store"], inplace=True)

In [306]:
smaller_clothes_df = clothes_df[["join_name", "product_url", "store", "name", "price", "description", "brand"]]

In [310]:
merged_df = pd.merge(df.reset_index(), smaller_clothes_df, on="join_name")

In [312]:
merged_df.drop(columns=["recommendation"], inplace=True)

In [346]:
def cleaned_price(row):
    cleaned_price = str(row["price"])
    price_list = str(row["price"]).lower().split()
    if price_list[0] == "now" or price_list[0] == "sale":
        cleaned_price = price_list[1]
    elif price_list[0] == "the" and '$' in price_list[3]:
        cleaned_price = price_list[3]
        
    if cleaned_price[-1] == ".":
        cleaned_price = cleaned_price[:-1]
    return cleaned_price

df_merged["cleaned_price"] = df_merged.apply(cleaned_price, axis=1)

In [347]:
df_merged["cleaned_price"] = df_merged["cleaned_price"].str.replace("$", "")
df_merged["cleaned_price"] = df_merged["cleaned_price"].str.replace("nan", "0")
df_merged["cleaned_price"] = df_merged["cleaned_price"].str.replace(",", "")
df_merged["cleaned_price"] = df_merged["cleaned_price"].str.replace("orig", "")
df_merged["cleaned_price"] = df_merged["cleaned_price"].str.replace(",", "")

In [348]:
df_merged["cleaned_price"] = df_merged["cleaned_price"].astype(float)

In [351]:
df_merged.to_pickle("complete_clothes.pkl")